# D&D Battle Sim
Basically running encounter simulations in python. Import two stat block, run battle scenarios, get the outcome. We're doing this without a graphical interface for the moment.

In [8]:
# Imports
import requests
import json

Running a basic rat vs commoner test here.

In [13]:
# importing stat blocks
baseURL = "https://www.dnd5eapi.co/api/monsters/"

commoner = json.loads(requests.get(baseURL + "commoner").text)
rat = json.loads(requests.get(baseURL + "rat").text)
